In [3]:
import cv2
import matplotlib.pyplot as plt
import importlib
import os
from ipywidgets import interact
from ipywidgets import interact_manual
from IPython.display import display
from ipywidgets import widgets

In [4]:
import ipynb.fs.full.Bios.wspFFA as wspFirefly
import ipynb.fs.full.wspCS as wspCuckooSearch
import ipynb.fs.full.wspKH as wspKrillHerd
import ipynb.fs.full.wspEHO as wspElephantHerding
import ipynb.fs.full.wspABC as wspArtificialBeeColony

import ipynb.fs.full.plotGraph as plotGraph
import ipynb.fs.full.wspMultithreshold as wspMultithreshold
import ipynb.fs.full.dicomHandler as dicomHandler
import ipynb.fs.full.npyHandler as npyHandler

import ipynb.fs.full.wspTsallisEvaluation as wspTsallisEvaluation
import ipynb.fs.full.wspShannonEvaluation as wspShannonEvaluation

import ipynb.fs.full.automaticQCalculation as automaticQCalculation

importlib.reload(wspFirefly)
importlib.reload(wspCuckooSearch)
importlib.reload(wspKrillHerd)
importlib.reload(wspElephantHerding)
importlib.reload(wspArtificialBeeColony)

importlib.reload(plotGraph)
importlib.reload(wspMultithreshold)
importlib.reload(dicomHandler)
importlib.reload(wspTsallisEvaluation)

importlib.reload(automaticQCalculation)

ModuleNotFoundError: No module named 'ipynb.fs.full.Bios'

In [ ]:
def readImage(img_path):
    image = cv2.imread(img_path)
    img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    return img

In [ ]:
bio_algorithms = ['FFA','KH', 'CS', 'ABC', 'EHO']

In [ ]:
folder_dir = "Dataset/original"
folder_gt_dir = "Dataset/segmented"
all_images = ['CASE_20_1.npy', 'CASE_24_10.npy']

for file in os.listdir(folder_dir):
    all_images.append(file)

In [ ]:
def main_dicom(image, algorithm, dimension, q):
    dicom_image, pixel_array = dicomHandler.read_dicom_image(image)
    print("Original:", pixel_array.min(), pixel_array.max())
    transformed_pixel_array = dicomHandler.image_transformation(pixel_array)
    print("Transformação:", transformed_pixel_array.min(), transformed_pixel_array.max())
    
    plotGraph.plot_2_images(pixel_array, transformed_pixel_array)

    original_image = pixel_array.copy()

    hu_image = dicomHandler.transform_to_hu(dicom_image, transformed_pixel_array)

    hist, bin_edges, best_thresholds, img_thres = wspMultithreshold.wspMultithreshold(hu_image, algorithm, dimension, q)

    high_intensity = wspMultithreshold.get_high_intensity_pixels(img_thres)
    
    pixel_array = dicomHandler.transform_to_pixel_array(dicom_image, high_intensity)

    hard_tissue = wspMultithreshold.get_largests_regions(pixel_array, original_image)

    #dicomHandler.save_dicom(dicom_image, pixel_array,"images/temp.dcm")
    
    plotGraph.plot_image_histogram(hu_image, hist, bin_edges)
    plotGraph.plot_image_histogram_threshold(img_thres, hist, bin_edges, best_thresholds)
    dicomHandler.show_dicom_image(high_intensity)
    dicomHandler.show_dicom_image(hard_tissue)

In [ ]:
def main_npy(image, gamma, bio, limiar):
    npy_img = folder_dir + '/' + image

    q = automaticQCalculation.get_best_q(npy_img)

    automaticQCalculation.plot_q_curve()

    pixel_array = npyHandler.read_npy_image(npy_img)

    original_image = pixel_array.copy()

    transformed_pixel_array = npyHandler.image_transformation(pixel_array, gamma)

    #Show results
    plotGraph.plot_n_images([original_image, transformed_pixel_array] , ["Original", "Transformação Gama"])

    hist, bin_edges, best_thresholds, img_thres = wspMultithreshold.wspMultithreshold(transformed_pixel_array, bio, limiar, q)

    # Show results
    plotGraph.plot_image_histogram_threshold(img_thres, hist, bin_edges, best_thresholds)

    high_intensity = wspMultithreshold.get_high_intensity_pixels(img_thres)

    pixel_array = npyHandler.transform_to_npy_pixel_array(high_intensity)

    hard_tissue = wspMultithreshold.get_largests_regions(pixel_array, original_image)

    gt_image = npyHandler.read_npy_image(folder_gt_dir + '/' + image)
    plotGraph.plot_n_images([high_intensity, hard_tissue, gt_image] , ["Maior intensidade", "Maior região", "Padrão Ouro"])


In [ ]:
def main_normal_image(image, gamma, bio, limiar, q):
    img = readImage(f"{image}")
    
    hist, bin_edges, best_thresholds, img_thres = wspMultithreshold.wspMultithreshold(img, bio, limiar, q)

    plotGraph.plot_image_histogram_threshold(img_thres, hist, bin_edges, best_thresholds)

In [ ]:
interact_manual(main_npy,
    image=all_images,
    gamma=widgets.FloatSlider(min=1, max=2, step=0.1, value=1.3),
    bio=bio_algorithms,
    limiar=widgets.IntSlider(min=1, max=5, step=1, value=2))

interactive(children=(Dropdown(description='image', options=('CASE_20_1.npy', 'CASE_24_10.npy', 'CASE_10_1.npy…

<function __main__.main_npy(image, gamma, bio, limiar)>